In [1]:
import pandas as pd


import numpy as np
import pandas as pd

import seaborn as sns
from importlib import reload
import matplotlib.pyplot as plt
import preprocess_data 

reload(preprocess_data)

%matplotlib inline

In [2]:
inpatient_specialty = pd.read_csv('/data/public/MLA/VCHAMPS-Quality/inpatient_specialty_qual.csv').iloc[:,1:]

In [3]:
inpatient_specialty['Specialty start date'] = inpatient_specialty['Specialty start date'].apply(preprocess_data.preprocess_time_string)
inpatient_specialty['Specialty end date'] = inpatient_specialty['Specialty end date'].apply(preprocess_data.preprocess_time_string)
inpatient_specialty['period'] = preprocess_data.get_period(inpatient_specialty['Specialty start date'],inpatient_specialty['Specialty end date'])

In [4]:
specialty_class = preprocess_data.generate_dict_from_csv('/home/vivi/FDA_datasets/specialty_class.csv','Specialty','Class')

In [5]:
inpatient_specialty['Specialty encoded'] = preprocess_data.preprocess_substitute_categories(specialty_class, 
                                                                               inpatient_specialty['Specialty'])

### Merge with inpatient data
Filter out specialty time before last discharge date

In [6]:
path = '/home/hassan/lily/MLA/FDA/outpatient_mortality_quality.csv'
inpatient = pd.read_csv(path).iloc[:,1:]
inpatient.head()

,Internalpatientid,num_visits,CVD,last_visit_date,Age 00-20,Age 20-40,Age 40-60,Age 60-80,Age 80-100,Age 100-120,age_mean,age_std,freq
0,67,313,0,2020-11-11 16:00:31.0,0,0,313,0,0,0,53.523105,3.128784,28.45
1,200,1872,1,2023-02-13 10:31:19.0,0,0,0,741,1131,0,80.476890,4.781990,81.39
2,291,234,0,2007-03-26 14:15:45.0,0,0,0,0,234,0,81.506487,1.264804,58.50
3,330,932,1,2024-03-01 00:19:37.0,0,0,51,881,0,0,69.622150,4.492263,40.52
4,351,773,1,2024-06-04 14:45:34.0,0,0,0,0,773,0,85.033989,0.541659,154.60


In [7]:
inpatient_date = inpatient[['Internalpatientid','last_visit_date']]
inpatient_date = inpatient_date.drop_duplicates(keep = 'first')
inpatient_date

,Internalpatientid,last_visit_date
0,67,2020-11-11 16:00:31.0
1,200,2023-02-13 10:31:19.0
2,291,2007-03-26 14:15:45.0
3,330,2024-03-01 00:19:37.0
4,351,2024-06-04 14:45:34.0
...,...,...
995,167907,2013-11-14 23:44:48.0
996,167917,2022-01-27 05:25:51.0
997,168008,2013-12-03 23:38:48.0
998,168496,2023-02-03 22:36:15.0


In [8]:
inpatient_specialty2 = inpatient_specialty.merge(inpatient_date, how = 'left', on = 'Internalpatientid')
inpatient_specialty3 = inpatient_specialty2.loc[inpatient_specialty2['Specialty end date'] <= inpatient_specialty2['last_visit_date']]
inpatient_specialty3

,Internalpatientid,Age at specialty,Specialty start date,Specialty end date,Specialty,State,period,Specialty encoded,last_visit_date
0,100399,84.470360,2008-12-27 22:00:08,2009-01-01 01:41:45,"PULMONARY, NON-TB",Minnesota,4 days 03:41:37,NaN,2013-02-07 02:41:24.0
1,100399,85.551411,2010-01-27 00:58:53,2010-02-03 20:55:56,SUBSTANCE ABUSE TRMT UNIT,Minnesota,7 days 19:57:03,1.0,2013-02-07 02:41:24.0
2,100399,86.335481,2010-11-09 14:50:30,2010-11-10 17:47:52,SURGICAL OBSERVATION,Minnesota,1 days 02:57:22,0.0,2013-02-07 02:41:24.0
3,100694,72.493945,2004-08-04 12:53:09,2004-08-07 14:21:19,SURGICAL STEPDOWN,Idaho,3 days 01:28:10,0.0,2022-12-29 18:45:55.0
4,102468,59.676888,2011-04-08 21:39:44,2011-04-12 23:58:20,GEM ACUTE MEDICINE,North Carolina,4 days 02:18:36,0.0,2021-07-14 16:09:34.0
...,...,...,...,...,...,...,...,...,...
11384,98416,73.041255,2000-09-25 13:53:23,2000-09-26 05:09:15,HALFWAY HOUSE,California,0 days 15:15:52,1.0,2007-01-12 20:00:53.0
11385,98416,78.018555,2005-09-18 18:45:38,2005-09-18 18:45:38,CARDIOLOGY,California,0 days 00:00:00,0.0,2007-01-12 20:00:53.0
11386,99024,68.288449,2014-08-25 10:16:36,2014-08-25 10:16:36,GENERAL(ACUTE MEDICINE),Oregon,0 days 00:00:00,0.0,2015-09-05 15:23:58.0
11387,9929,72.785598,2011-11-05 03:15:59,2011-11-07 03:15:58,ANESTHESIOLOGY,Arizona,1 days 23:59:59,0.0,2014-12-11 11:23:09.0


In [9]:
inpatient_specialty3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11334 entries, 0 to 11388
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype          
---  ------                --------------  -----          
 0   Internalpatientid     11334 non-null  int64          
 1   Age at specialty      11334 non-null  float64        
 2   Specialty start date  11334 non-null  datetime64[ns] 
 3   Specialty end date    11334 non-null  datetime64[ns] 
 4   Specialty             11334 non-null  object         
 5   State                 11334 non-null  object         
 6   period                11334 non-null  timedelta64[ns]
 7   Specialty encoded     10859 non-null  float64        
 8   last_visit_date       11334 non-null  object         
dtypes: datetime64[ns](2), float64(2), int64(1), object(3), timedelta64[ns](1)
memory usage: 885.5+ KB


In [10]:
inpatient_specialty3['specialty count'] = inpatient_specialty3['Specialty']\
    .groupby(inpatient_specialty3['Internalpatientid']).transform('count')

specialty_medical_count = inpatient_specialty3.groupby('Internalpatientid')['Specialty encoded']\
    .apply(lambda x: (x == 0).sum()).reset_index(name='specialty medical count')

specialty_support_count = inpatient_specialty3.groupby('Internalpatientid')['Specialty encoded']\
    .apply(lambda x: (x == 1).sum()).reset_index(name='specialty support count')

inpatient_specialty3['mean age at specailty'] = inpatient_specialty3['Age at specialty']\
    .groupby(inpatient_specialty3['Internalpatientid']).transform('mean')

/tmp/ipykernel_284800/3919990082.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inpatient_specialty3['specialty count'] = inpatient_specialty3['Specialty']\


/tmp/ipykernel_284800/3919990082.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inpatient_specialty3['mean age at specailty'] = inpatient_specialty3['Age at specialty']\


In [11]:
period_mean = inpatient_specialty.groupby('Internalpatientid')['period'].agg(lambda x: x).reset_index()
period_mean['period'] = period_mean['period'].apply(lambda x: np.nanmean(x.total_seconds()/86400))

period_std = inpatient_specialty.groupby('Internalpatientid')['period'].agg(lambda x: x).reset_index()
period_std['period'] = period_std['period'].apply(lambda x: np.nanstd(x.total_seconds()/86400,ddof=0))

period_mean = period_mean.rename(columns={'period': 'period mean'})
period_std = period_std.rename(columns={'period': 'period std'})

In [12]:
inpatient_specialty4 = inpatient_specialty3[['Internalpatientid', 'mean age at specailty','specialty count']].sort_values('Internalpatientid')
inpatient_specialty4 = inpatient_specialty4.groupby('Internalpatientid').apply(lambda x:x)
inpatient_specialty4 = inpatient_specialty4.drop_duplicates().reset_index(drop = True)
inpatient_specialty4

/tmp/ipykernel_284800/554132651.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  inpatient_specialty4 = inpatient_specialty4.groupby('Internalpatientid').apply(lambda x:x)


,Internalpatientid,mean age at specailty,specialty count
0,67,56.998234,2
1,200,79.664650,15
2,291,82.644124,23
3,330,70.947996,15
4,351,84.981808,16
...,...,...,...
626,166881,75.529507,4
627,167102,68.142469,9
628,167404,77.965193,3
629,167917,45.426367,14


In [13]:
inpatient_specialty5 = inpatient_specialty4.merge(period_mean, how = 'left', on = 'Internalpatientid')
inpatient_specialty5 = inpatient_specialty5.merge(period_std, how = 'left', on = 'Internalpatientid')

inpatient_specialty5 = inpatient_specialty5.merge(specialty_medical_count, how = 'left', on = 'Internalpatientid')
inpatient_specialty5 = inpatient_specialty5.merge(specialty_support_count, how = 'left', on = 'Internalpatientid')

inpatient_specialty5 = inpatient_specialty5.dropna(subset = 'period mean')
inpatient_specialty5

,Internalpatientid,mean age at specailty,specialty count,period mean,period std,specialty medical count,specialty support count
0,67,56.998234,2,0.089728,0.089728,2,0
1,200,79.664650,15,1.771890,3.276867,9,6
2,291,82.644124,23,3.711998,5.827537,7,15
3,330,70.947996,15,1.829252,1.986030,14,1
4,351,84.981808,16,2.304762,2.520672,4,12
...,...,...,...,...,...,...,...
626,166881,75.529507,4,1.937271,2.161738,4,0
627,167102,68.142469,9,1.591543,1.560624,6,3
628,167404,77.965193,3,3.875710,3.145414,2,0
629,167917,45.426367,14,0.803098,0.899967,13,0


In [14]:
# inpatient_specialty5.to_csv('/home/vivi/FDA_datasets/inpatient_specialty_allcause_mortality_quality.csv')